### Determining new EV charger locations in Prague, Czech Republic
###### This is the main part of the analysis, based on the data from csv files no. 1, 2, 3, 4 - attached to the thesis.

In [117]:
import pandas as pd
pd.set_option("display.precision", 2)
pd.set_option('display.width', 200)

### Table of variables
###### Here is a list of all variables used in the analysis, some of which are used for the calculations below.

In [118]:
table = pd.read_csv("./Variables.csv", delimiter=";")
table

,Name,Value/Unit,Meaning,Determination/Calculation
0,xj,"{0,1}",Binary variable whether car park 𝑗 is selected...,"0,1"
1,nj,integer,Number of chargers in station 𝑗 per day,2
2,qj,integer,Number of cars charged by station 𝑗 at the sam...,4
3,pj,CZK,Charging price per minute in a station j,10
4,tj,minutes,Estimated duration of an EV being fully charged,81.6
5,mj,integer,Maximum number of charging sessions per day in...,71
6,lj,integer,Upper bound of chargers in station 𝑗,0.1 • pc
7,rij,"{0,1}",Binary variable which represents the service a...,"0,1"
8,fi,integer,Average traffic flow in grid cell 𝑖 per day,Individual value per tile
9,Ki,integer,Number of traffic flow measurement points in g...,Individual value per tile


Defining needed constants - converting them to a needed format.

In [119]:
table['Number_conversion'] = table['Determination/Calculation'].astype(float, errors='ignore')
table

,Name,Value/Unit,Meaning,Determination/Calculation,Number_conversion
0,xj,"{0,1}",Binary variable whether car park 𝑗 is selected...,"0,1","0,1"
1,nj,integer,Number of chargers in station 𝑗 per day,2,2
2,qj,integer,Number of cars charged by station 𝑗 at the sam...,4,4
3,pj,CZK,Charging price per minute in a station j,10,10
4,tj,minutes,Estimated duration of an EV being fully charged,81.6,81.6
5,mj,integer,Maximum number of charging sessions per day in...,71,71
6,lj,integer,Upper bound of chargers in station 𝑗,0.1 • pc,0.1 • pc
7,rij,"{0,1}",Binary variable which represents the service a...,"0,1","0,1"
8,fi,integer,Average traffic flow in grid cell 𝑖 per day,Individual value per tile,Individual value per tile
9,Ki,integer,Number of traffic flow measurement points in g...,Individual value per tile,Individual value per tile


In [120]:
def get_variable_value(table: pd.DataFrame, variable_name: str):
    try:
        value = float(table[table['Name'] == variable_name]['Number_conversion'])
    except:
        value = table[table['Name'] == variable_name]['Number_conversion']
    return value

get_variable_value(table, 'mj')

71.0

#### Loading the file with the key data for the scoring model. 


In [121]:
table_2 = pd.read_csv("./File 4 - New charger placement scoring model.csv", delimiter=";")
table_2.astype(float)

,Grid ID,No. of charging stations in grid,No. of car parks in grid,pc,fi
0,1.0,0.0,0.0,0.0,0.0
1,2.0,0.0,0.0,0.0,0.0
2,3.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0
4,5.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
377,378.0,0.0,1.0,35.0,15700.0
378,379.0,0.0,0.0,0.0,0.0
379,380.0,0.0,0.0,0.0,0.0
380,381.0,0.0,1.0,36.0,21800.0


#### Here are the calculation of the needed variables to determine which grids (Grid ID) have the highest missing demand (dri)


In [122]:
table_2['di'] = get_variable_value(table, 'u') * table_2['fi']
table_2

,Grid ID,No. of charging stations in grid,No. of car parks in grid,pc,fi,di
0,1,0,0,0.0,0.0,0.00
1,2,0,0,0.0,0.0,0.00
2,3,0,0,0.0,0.0,0.00
3,4,0,0,0.0,0.0,0.00
4,5,0,0,0.0,0.0,0.00
...,...,...,...,...,...,...
377,378,0,1,35.0,15700.0,77.31
378,379,0,0,0.0,0.0,0.00
379,380,0,0,0.0,0.0,0.00
380,381,0,1,36.0,21800.0,107.34


In [123]:
table_2['diz'] = get_variable_value(table, 'mj') * table_2['No. of charging stations in grid']
table_2

,Grid ID,No. of charging stations in grid,No. of car parks in grid,pc,fi,di,diz
0,1,0,0,0.0,0.0,0.00,0.0
1,2,0,0,0.0,0.0,0.00,0.0
2,3,0,0,0.0,0.0,0.00,0.0
3,4,0,0,0.0,0.0,0.00,0.0
4,5,0,0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...
377,378,0,1,35.0,15700.0,77.31,0.0
378,379,0,0,0.0,0.0,0.00,0.0
379,380,0,0,0.0,0.0,0.00,0.0
380,381,0,1,36.0,21800.0,107.34,0.0


In [124]:
table_2['dri'] = table_2['di'] - table_2['diz']
table_2

,Grid ID,No. of charging stations in grid,No. of car parks in grid,pc,fi,di,diz,dri
0,1,0,0,0.0,0.0,0.00,0.0,0.00
1,2,0,0,0.0,0.0,0.00,0.0,0.00
2,3,0,0,0.0,0.0,0.00,0.0,0.00
3,4,0,0,0.0,0.0,0.00,0.0,0.00
4,5,0,0,0.0,0.0,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...
377,378,0,1,35.0,15700.0,77.31,0.0,77.31
378,379,0,0,0.0,0.0,0.00,0.0,0.00
379,380,0,0,0.0,0.0,0.00,0.0,0.00
380,381,0,1,36.0,21800.0,107.34,0.0,107.34


In [125]:
table_2['dic'] = table_2['pc'] * 17.65
table_2

,Grid ID,No. of charging stations in grid,No. of car parks in grid,pc,fi,di,diz,dri,dic
0,1,0,0,0.0,0.0,0.00,0.0,0.00,0.00
1,2,0,0,0.0,0.0,0.00,0.0,0.00,0.00
2,3,0,0,0.0,0.0,0.00,0.0,0.00,0.00
3,4,0,0,0.0,0.0,0.00,0.0,0.00,0.00
4,5,0,0,0.0,0.0,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...
377,378,0,1,35.0,15700.0,77.31,0.0,77.31,617.75
378,379,0,0,0.0,0.0,0.00,0.0,0.00,0.00
379,380,0,0,0.0,0.0,0.00,0.0,0.00,0.00
380,381,0,1,36.0,21800.0,107.34,0.0,107.34,635.40


#### Here is an adjustment to the variable 'Demand covered by new chargers' not to show inf, but 0 as a lowest possible value.

In [126]:
table_2['Demand covered by new chargers'] = get_variable_value(table, 'mj') / table_2['dri']
table_2

,Grid ID,No. of charging stations in grid,No. of car parks in grid,pc,fi,di,diz,dri,dic,Demand covered by new chargers
0,1,0,0,0.0,0.0,0.00,0.0,0.00,0.00,inf
1,2,0,0,0.0,0.0,0.00,0.0,0.00,0.00,inf
2,3,0,0,0.0,0.0,0.00,0.0,0.00,0.00,inf
3,4,0,0,0.0,0.0,0.00,0.0,0.00,0.00,inf
4,5,0,0,0.0,0.0,0.00,0.0,0.00,0.00,inf
...,...,...,...,...,...,...,...,...,...,...
377,378,0,1,35.0,15700.0,77.31,0.0,77.31,617.75,0.92
378,379,0,0,0.0,0.0,0.00,0.0,0.00,0.00,inf
379,380,0,0,0.0,0.0,0.00,0.0,0.00,0.00,inf
380,381,0,1,36.0,21800.0,107.34,0.0,107.34,635.40,0.66


In [127]:
table_2.replace([float('inf')], 0, inplace=True)
table_2

,Grid ID,No. of charging stations in grid,No. of car parks in grid,pc,fi,di,diz,dri,dic,Demand covered by new chargers
0,1,0,0,0.0,0.0,0.00,0.0,0.00,0.00,0.00
1,2,0,0,0.0,0.0,0.00,0.0,0.00,0.00,0.00
2,3,0,0,0.0,0.0,0.00,0.0,0.00,0.00,0.00
3,4,0,0,0.0,0.0,0.00,0.0,0.00,0.00,0.00
4,5,0,0,0.0,0.0,0.00,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
377,378,0,1,35.0,15700.0,77.31,0.0,77.31,617.75,0.92
378,379,0,0,0.0,0.0,0.00,0.0,0.00,0.00,0.00
379,380,0,0,0.0,0.0,0.00,0.0,0.00,0.00,0.00
380,381,0,1,36.0,21800.0,107.34,0.0,107.34,635.40,0.66


#### This is the final table which is sorted by descending order to show grids with highest outstanding demand not covered by the existing chargers.
###### Since we look for 10 chargers, the top 10 grids where is an available car park for building ('No. of car parks in grid > 0), the selected grids are:
###### Grid ID: 289, 272, 14, 267, 225, 32, 18, 141, 42, 145

In [132]:
#table_2['Demand covered by new chargers with unrestricted lj'] = table_2['Demand covered by new chargers'] / table_2['dri']
#table_2['Demand covered by new chargers with unrestricted lj'] = table_2['Demand covered by new chargers with unrestricted lj'].fillna(0) 
#table_2['Demand covered by new chargers with unrestricted lj'] = table_2['Demand covered by new chargers with unrestricted lj'].astype(float) 
#table_2

table_2.sort_values(by = 'dri', ascending = False, inplace=True)
table_2.head(40)

,Grid ID,No. of charging stations in grid,No. of car parks in grid,pc,fi,di,diz,dri,dic,Demand covered by new chargers
147,148,0,0,0.0,132450.00,652.19,0.0,652.19,0.00,0.11
249,250,0,0,0.0,125500.00,617.96,0.0,617.96,0.00,0.11
288,289,0,1,26.5,121900.00,600.24,0.0,600.24,467.72,0.12
324,325,0,0,0.0,117600.00,579.06,0.0,579.06,0.00,0.12
128,129,0,0,0.0,113000.00,556.41,0.0,556.41,0.00,0.13
25,26,0,0,0.0,90300.00,444.64,0.0,444.64,0.00,0.16
198,199,0,0,0.0,87900.00,432.82,0.0,432.82,0.00,0.16
292,293,0,0,0.0,83100.00,409.19,0.0,409.19,0.00,0.17
63,64,0,0,0.0,83000.00,408.69,0.0,408.69,0.00,0.17
271,272,0,1,14.5,82300.00,405.25,0.0,405.25,255.92,0.18
